In [1]:
# Funciones importadas

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn import tree
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import requests
import lxml.html as lh
from bs4 import BeautifulSoup
import numpy as np

from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier       
from sklearn import metrics
from sklearn.metrics import accuracy_score

from imblearn.over_sampling import SMOTE, ADASYN, SVMSMOTE


### Lectura de csv

In [2]:
# Función de lectura de csv para añadir la funcionalidad de que ponga la columna nombre como index

def read_file(file):
    file = pd.read_csv(file)
    file.set_index(('Name'), inplace=True)
    return file

# Importación de DataFrames

all_star_college = read_file('../input/model_dummies_allstar.csv')
draft19_class = read_file('../input/model_draft19_class.csv')


### Aplicación del modelo

In [66]:
# Función para aplicar modelos de machine learning y determinar su accuracy y su confusion matrix.

def modelos(data,columnadep):
    
    X=data.loc[:,data.columns!= columnadep]
    y=data[columnadep]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # Definición de modelos                                          # Modelos aplicados: 
    
    cls = svm.SVC(gamma='auto', probability=True)                    # 1. Linear regresion
    lr = LogisticRegression(solver ='liblinear',max_iter=500)        # 2. Logistic regresion
    neigh3 = KNeighborsClassifier(n_neighbors=3)                     # 3. k-Neighbours k=3 
    neigh5 = KNeighborsClassifier(n_neighbors=5)                     # 4. k-Neighbours k=5 
    rf=RandomForestClassifier(n_estimators=500, criterion='gini')    # 5. RandomForest 
    gnb = GaussianNB()                                               # 6. Gaussian Method
    svc = SVC(kernel='rbf', gamma='scale')                           # 7. SVC
    gbc=GradientBoostingClassifier()                                 # 8. GradientBoostingClassifier
    
    modelos=[cls, lr, neigh3, neigh5, rf, gnb, svc, gbc]
        
    for modelo in modelos:
        modelo.fit(X_train, y_train)
        y_predict = modelo.predict(X_test)
        print('{}'.format(modelo.__class__.__name__))
        print('Accuracy:',"\n", metrics.accuracy_score(y_test, y_predict))
        print("confusion matrix:","\n",confusion_matrix(y_test, y_predict))
        print('---------------------')


In [67]:
modelos(all_star_college, 'ALLSTAR_Yes')

SVC
Accuracy: 
 0.9043478260869565
confusion matrix: 
 [[104   0]
 [ 11   0]]
---------------------
LogisticRegression
Accuracy: 
 0.9130434782608695
confusion matrix: 
 [[104   0]
 [ 10   1]]
---------------------
KNeighborsClassifier
Accuracy: 
 0.8956521739130435
confusion matrix: 
 [[102   2]
 [ 10   1]]
---------------------
KNeighborsClassifier
Accuracy: 
 0.9130434782608695
confusion matrix: 
 [[104   0]
 [ 10   1]]
---------------------
RandomForestClassifier
Accuracy: 
 0.9043478260869565
confusion matrix: 
 [[104   0]
 [ 11   0]]
---------------------
GaussianNB
Accuracy: 
 0.8260869565217391
confusion matrix: 
 [[93 11]
 [ 9  2]]
---------------------
SVC
Accuracy: 
 0.9043478260869565
confusion matrix: 
 [[104   0]
 [ 11   0]]
---------------------
GradientBoostingClassifier
Accuracy: 
 0.8956521739130435
confusion matrix: 
 [[103   1]
 [ 11   0]]
---------------------


In [5]:
# Usaremos RandomForest al ser uno de los que mejor accuracy tiene

### Aplicación de over sampling

### Aplicación de over sampling  - creación de valores sinteticos

In [7]:
# Nuestras matrices de confusión nos indican que tenemos datos descompensados, por tanto debemos generar datos sinteticos para entrenar al modelo
# Usamos RandomForest para generar over sampling y así obtener un DataFrame con todos los valores sinteticos

X = all_star_college
y = all_star_college['ALLSTAR_Yes']

X_resample, y_resample = SVMSMOTE().fit_resample(X, y)
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_resample,
                                                            y_resample,
                                                            test_size = 0.20, 
                                                            random_state = 10)

rf=RandomForestClassifier(n_estimators=500, criterion='gini')
rf.fit(X, y)

# Aplicamos el modelo

y_pred_over = rf.predict(X_test_r)

over_samplingDF = pd.DataFrame(X_test_r)
over_samplingDF.head()

print('Modelo:')
display(over_samplingDF.head())
print('Shape del modelo:')
display(over_samplingDF.shape)

Modelo:


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,28.700000,8.600000,0.453000,2.800000,0.287000,6.100000,0.707000,7.900000,1.300000,0.600000,0.300000,2.400000,2.900000,14.400000,0.0,1.000000,0.0
1,26.714309,4.894941,0.515522,2.363874,0.299513,2.140316,0.728723,4.435257,3.631937,1.577312,0.500000,2.100000,2.122688,8.749565,1.0,0.000000,1.0
2,23.100000,5.900000,0.639000,0.200000,0.200000,5.100000,0.685000,8.100000,1.400000,0.900000,2.600000,2.400000,3.400000,11.200000,1.0,0.000000,0.0
3,32.200000,9.400000,0.579000,0.200000,0.000000,4.500000,0.564000,8.400000,0.800000,0.600000,1.500000,2.500000,2.100000,13.400000,0.0,0.000000,0.0
4,24.724742,6.182032,0.459430,2.368484,0.502955,3.124742,0.713739,7.482032,3.371871,1.317968,0.446097,2.017968,2.778645,10.578645,1.0,1.393226,0.0


Shape del modelo:


(206, 17)

In [8]:
# Aplicamos las columnas que tiene el DataFrame 'top5_college' al nuevo DataFrame

colheaders = all_star_college.columns   
    
dictiona = {x: y for x, y in zip(range(18), colheaders)}

over_samplingDF = over_samplingDF.rename(index = str, columns=dictiona)
over_samplingDF.head()

,MP,2PA,2P%,3PA,3P%,FTA,FT%,TRB,AST,STL,BLK,TOV,PF,PTS,ALLSTAR_Yes,POS_Forward,POS_Guard
0,28.700000,8.600000,0.453000,2.800000,0.287000,6.100000,0.707000,7.900000,1.300000,0.600000,0.300000,2.400000,2.900000,14.400000,0.0,1.000000,0.0
1,26.714309,4.894941,0.515522,2.363874,0.299513,2.140316,0.728723,4.435257,3.631937,1.577312,0.500000,2.100000,2.122688,8.749565,1.0,0.000000,1.0
2,23.100000,5.900000,0.639000,0.200000,0.200000,5.100000,0.685000,8.100000,1.400000,0.900000,2.600000,2.400000,3.400000,11.200000,1.0,0.000000,0.0
3,32.200000,9.400000,0.579000,0.200000,0.000000,4.500000,0.564000,8.400000,0.800000,0.600000,1.500000,2.500000,2.100000,13.400000,0.0,0.000000,0.0
4,24.724742,6.182032,0.459430,2.368484,0.502955,3.124742,0.713739,7.482032,3.371871,1.317968,0.446097,2.017968,2.778645,10.578645,1.0,1.393226,0.0


### Aplicación de Gridsearch

In [9]:
# Hacemos un gridsearch para obtener los parámetros óptimos para el modelo de RandomForest

X=over_samplingDF.loc[:,over_samplingDF.columns!= 'ALLSTAR_Yes']
y=over_samplingDF['ALLSTAR_Yes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

rfc=RandomForestClassifier(random_state=42)

param_grid = {
   'n_estimators': [200, 500],
   'max_features': ['auto', 'sqrt'],
   'max_depth' : [7,8],
   'criterion' :['gini', 'entropy']}

fitting = rf.fit(X_test,y_test)

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)
CV_rfc.best_params_

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'criterion': 'entropy',
 'max_depth': 8,
 'max_features': 'auto',
 'n_estimators': 200}

### Gridsearch aplicado a RF de over sampling

In [10]:
# Con los parámetros obtenidos en el Gridsearch entrenamos el modelo

X=over_samplingDF.loc[:,over_samplingDF.columns!= 'ALLSTAR_Yes']
y=over_samplingDF['ALLSTAR_Yes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

rf=RandomForestClassifier(criterion='entropy', max_depth=8, max_features='auto', n_estimators=500)
rf.fit(X_train, y_train) 

y_pred_rf= rf.predict(X_test)

print("Accuracy rf:",metrics.accuracy_score(y_test, y_pred_rf))
print("Confusion matrix rf","\n",confusion_matrix(y_test, y_pred_rf))

Accuracy rf: 0.8095238095238095
Confusion matrix rf 
 [[12  7]
 [ 1 22]]


### Aplicado al draft de 2019

In [11]:
prediction = rf.predict(draft19_class)

In [12]:
# Obtenemos la lista de los jugadores que nuestro modelo predice que serán All Star durante su carrera

draft19_class['Prediction'] = prediction
draft19_class.loc[draft19_class['Prediction']==1]

,MP,2PA,2P%,3PA,3P%,FTA,FT%,TRB,AST,STL,BLK,TOV,PF,PTS,POS_Forward,POS_Guard,Prediction
Name,,,,,,,,,,,,,,,,,
KY BOWMAN,35.5,7.8,0.466,5.8,0.388,3.9,0.762,6.4,3.9,1.3,0.3,3.0,2.6,16.9,0,1,1.0
BRUNO FERNANDO,26.4,7.4,0.603,0.2,0.308,3.8,0.763,8.7,1.4,0.5,1.6,2.3,2.8,12.0,1,0,1.0
JA MORANT,35.3,8.9,0.545,3.8,0.343,6.3,0.810,6.1,8.2,1.4,0.6,3.8,1.7,18.7,0,1,1.0
SHAMORIE PONDS,35.2,9.1,0.507,5.9,0.328,5.2,0.840,4.5,4.3,2.3,0.2,2.2,1.8,19.5,0,1,1.0
D’MARCUS SIMONDS,32.9,10.3,0.520,4.0,0.294,5.3,0.690,5.0,3.4,1.4,0.7,3.4,2.4,17.9,0,1,1.0
RAYJON TUCKER,36.6,6.9,0.558,5.8,0.411,7.0,0.777,6.7,1.8,1.1,0.4,2.6,2.3,20.3,0,1,1.0
ZION WILLIAMSON,30.0,11.0,0.747,2.2,0.338,6.2,0.640,8.9,2.1,2.1,1.8,2.4,2.1,22.6,1,0,1.0
